# Optimization with JuMP

JulaOpt is a collection of Julia optimization tools, and it's one of the best parts of the Julia ecosystem. Some of the packages in JuliaOpt are:

![](img/juliaopt.png)

For today, we're just going to show off JuMP, a modeling tool designed to make it easy to efficiently specify and solve optimizations in Julia. JuMP is similar to `yalmip` (in MATLAB), `pyomo` (in Python), or AMPL. It distinguishes itself by being fast and easy to use and taking advantage of Julia's expressiveness. 

In [1]:
using JuMP
using Ipopt

Creating a model in JuMP is easy:

In [2]:
model = Model(solver=IpoptSolver())

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is Ipopt

We add variables to the model with the `@variable` macro. Having a macro is useful because it can create variables inside the model and also, conveniently, create matching local variables in Julia:

In [3]:
@variable(model, x[1:2])

model

Feasibility problem with:
 * 0 linear constraints
 * 2 variables
Solver is Ipopt

In [4]:
typeof(x)

Array{JuMP.Variable,1}

Let's add a simple objective:

In [5]:
@objective(model, Min, sum(x.^2))
model

Minimization problem with:
 * 0 linear constraints
 * 2 variables
Solver is Ipopt

In [6]:
solve(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

And we can retrieve the solution:

In [7]:
getvalue(x)

2-element Array{Float64,1}:
 0.0
 0.0

That's a pretty boring model. Let's constrain the optimization:

In [8]:
@constraints model begin
    x[1] >= 2 * x[2]
    x[2] >= 1
end
model

Minimization problem with:
 * 2 linear constraints
 * 2 variables
Solver is Ipopt

In [9]:
solve(model)
getvalue(x)

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        3
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        2
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

2-element Array{Float64,1}:
 2.0
 1.0

That was easy. But it's still not a very interesting optimization. How about something more robotics-y? 

## Model-Predictive Control in JuMP

Let's implement a very simple model-predictive control (MPC) optimization in JuMP. Specifically, we'll write an optimization that tries to find a sequence of control inputs for a very simple model robot in order to optimize an objective function. 

More concretely, our model will be a brick, sliding frictionlessly in two dimensions. Our input will be the acceleration of the brick, and we'll try to minimize the brick's final velocity and distance from the origin. 

In [10]:
model = Model(solver=IpoptSolver(print_level=0))

# Define our constant parameters
Δt = 0.1
num_time_steps = 20
max_acceleration = 0.5

# Define our decision variables
@variables model begin
    position[1:2, 1:num_time_steps]
    velocity[1:2, 1:num_time_steps]
    -max_acceleration <= acceleration[1:2, 1:num_time_steps] <= max_acceleration
end

# Add dynamics constraints
@constraint(model, [i=2:num_time_steps, j=1:2],
            velocity[j, i] == velocity[j, i - 1] + acceleration[j, i - 1] * Δt)
@constraint(model, [i=2:num_time_steps, j=1:2],
            position[j, i] == position[j, i - 1] + velocity[j, i - 1] * Δt)

# Cost function: minimize final position and final velocity
@objective(model, Min, 
    100 * sum(position[:, end].^2) + sum(velocity[:, end].^2))

# Initial conditions:
@constraint(model, position[:, 1] .== [1, 0])
@constraint(model, velocity[:, 1] .== [0, -1])

solve(model)

:Optimal

In [11]:
# Extract the solution from the model
q = getvalue(position)
v = getvalue(velocity)
u = getvalue(acceleration)

2×20 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5  …  -0.5  -0.5  -0.5  -0.5  0.5  0.0
  0.5   0.5   0.5   0.5   0.5   0.5      0.5   0.5   0.5   0.5  0.5  0.0

### Drawing the Result

We can draw the output of the optimization using the `Plots.jl` package

In [12]:
using Plots
# Use the GR backend for Plots.jl, because it's fast
gr()

Plots.GRBackend()

In [13]:
# The @animate macro creates an animated plot, which lets us draw the
# optimized trajectory of the brick as a function of time
anim = @animate for i = 1:num_time_steps
    plot(q[1, :], q[2, :], xlim=(-1.1, 1.1), ylim=(-1.1, 1.1))
    plot!([q[1, i]], [q[2, i]], marker=(:hex, 6))
end

# The gif() function saves our animated plot to an animated gif
# Note: this may require you to have the `ffmpeg` program installed.
# On Ubuntu 16.04, you can get this with `sudo apt-get install ffmpeg`.
gif(anim, "img/mpc1.gif", fps = 30)

INFO: Saved animation to /Users/rdeits/Downloads/agile-test/packages/v0.6/AgileRoboticsTutorial/notebooks/img/mpc1.gif


Plots.AnimatedGif("/Users/rdeits/Downloads/agile-test/packages/v0.6/AgileRoboticsTutorial/notebooks/img/mpc1.gif")

## Running the MPC Controller

In a real application, we wouldn't just run the MPC optimization once. Instead, we might run the optimization at every time step using the robot's current state. 

To do that, let's wrap the MPC problem in a function called `run_mpc()` that takes the robot's current position and velocity as input:

In [14]:
solver = IpoptSolver(print_level=0)

# run_mpc() takes the robot's current position and velocity
# and returns an optimized trajectory of position, velocity, 
# and acceleration. 
function run_mpc(initial_position, initial_velocity)
    
    model = Model(solver=solver)

    Δt = 0.1
    num_time_steps = 10
    max_acceleration = 0.5

    @variables model begin
        position[1:2, 1:num_time_steps]
        velocity[1:2, 1:num_time_steps]
        -max_acceleration <= acceleration[1:2, 1:num_time_steps] <= max_acceleration
    end

    # Dynamics constraints
    @constraint(model, [i=2:num_time_steps, j=1:2],
                velocity[j, i] == velocity[j, i - 1] + acceleration[j, i - 1] * Δt)
    @constraint(model, [i=2:num_time_steps, j=1:2],
                position[j, i] == position[j, i - 1] + velocity[j, i - 1] * Δt)

    # Cost function: minimize final position and final velocity
    @objective(model, Min, 
        100 * sum(position[:, end].^2) + sum(velocity[:, end].^2))

    # Initial conditions:
    @constraint(model, position[:, 1] .== initial_position)
    @constraint(model, velocity[:, 1] .== initial_velocity)

    solve(model)
    return getvalue(position), getvalue(velocity), getvalue(acceleration)
end

run_mpc (generic function with 1 method)

We can demonstrate this by repeatedly running the MPC program, applying its planned acceleration to the brick, and then simulating one step forward in time:

In [15]:
# The robot's starting position and velocity
q = [1.0, 0.0]
v = [0.0, -1.0]

anim = @animate for i in 1:80
    # Plot the current position
    plot([q[1]], [q[2]], marker=(:hex, 10), xlim=(-1.1, 1.1), ylim=(-1.1, 1.1))
    
    # Run the MPC control optimization
    q_plan, v_plan, u_plan = run_mpc(q, v)
    
    # Draw the planned future states from the MPC optimization
    plot!(q_plan[1, :], q_plan[2, :], linewidth=5)
    
    # Apply the planned acceleration and simulate one step in time
    u = u_plan[:, 1]
    v += u * Δt
    q += v * Δt
end
gif(anim, "img/mpc2.gif")

INFO: Saved animation to /Users/rdeits/Downloads/agile-test/packages/v0.6/AgileRoboticsTutorial/notebooks/img/mpc2.gif


Plots.AnimatedGif("/Users/rdeits/Downloads/agile-test/packages/v0.6/AgileRoboticsTutorial/notebooks/img/mpc2.gif")